In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE DATABASE MASTERCLASS;
USE SCHEMA PUBLIC;
USE WAREHOUSE MASTERCLASSWH;

In [ ]:
-- Creating File Format if not exists

CREATE FILE FORMAT IF NOT EXISTS PARQUET_FILE_FORMAT
 TYPE = PARQUET 
 COMPRESSION = SNAPPY
 USE_VECTORIZED_SCANNER = TRUE;

-- CREATE STAGE IF NOT EXISTS DATA_STAGE
--  URL='s3://<bucket>/lineitems/'
--  STORAGE_INTEGRATION = MASTERCLASS
--  COMMENT = 'Lineitems data stage';

LIST @MASTERCLASS.PUBLIC.DATA_STAGE; -- S3 data

In [ ]:
CREATE OR REPLACE TABLE LINEITEMS (
    PK_ID               NUMBER(38, 0) NOT NULL IDENTITY(1,1) COMMENT 'Primary key',
    V                   VARIANT COMMENT 'This is the JSON data from the file',
    FILE_NAME           VARCHAR(500) COMMENT 'This is the name of the file that was loaded into S3 bucket',
    FILE_LAST_MODIFIED  TIMESTAMP_NTZ COMMENT 'This is when the file was loaded into S3 bucket',
    INSERTED_DATE       TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP() COMMENT 'This is when the record was inserted into the table',
    CONSTRAINT LINEITEMS_PK PRIMARY KEY (PK_ID)
) COMMENT = 'LINEITEMS table to load the data from S3 bucket';

In [ ]:
SELECT *
FROM @MASTERCLASS.PUBLIC.DATA_STAGE (FILE_FORMAT => PARQUET_FILE_FORMAT)
LIMIT 3;

In [ ]:
COPY INTO LINEITEMS (V, FILE_NAME, FILE_LAST_MODIFIED)
FROM (
	SELECT 
		 t.$1 AS V
		,METADATA$FILENAME AS FILE_NAME
		,METADATA$FILE_LAST_MODIFIED AS FILE_LAST_MODIFIED
	FROM @MASTERCLASS.PUBLIC.DATA_STAGE (file_format => PARQUET_FILE_FORMAT) t
)
PURGE = FALSE;

In [ ]:
SELECT COUNT(*) FROM LINEITEMS;

In [ ]:
SELECT NULLIF(V:"L_ORDERKEY"::VARCHAR, '')::VARCHAR      AS L_ORDERKEY
     , NULLIF(V:"L_PARTKEY"::VARCHAR, '')::VARCHAR       AS L_PARTKEY
     , NULLIF(V:"L_SUPPKEY"::VARCHAR, '')::VARCHAR       AS L_SUPPKEY
     , NULLIF(V:"L_LINENUMBER"::VARCHAR, '')::VARCHAR    AS L_LINENUMBER
     , NULLIF(V:"L_QUANTITY"::VARCHAR, '')::VARCHAR      AS L_QUANTITY
     , NULLIF(V:"L_EXTENDEDPRICE"::VARCHAR, '')::VARCHAR AS L_EXTENDEDPRICE
     , NULLIF(V:"L_DISCOUNT"::VARCHAR, '')::VARCHAR      AS L_DISCOUNT
     , NULLIF(V:"L_TAX"::VARCHAR, '')::VARCHAR           AS L_TAX
     , NULLIF(V:"L_RETURNFLAG"::VARCHAR, '')::VARCHAR    AS L_RETURNFLAG
     , NULLIF(V:"L_LINESTATUS"::VARCHAR, '')::VARCHAR    AS L_LINESTATUS
     , NULLIF(V:"L_SHIPDATE"::VARCHAR, '')::VARCHAR      AS L_SHIPDATE
     , NULLIF(V:"L_COMMITDATE"::VARCHAR, '')::VARCHAR    AS L_COMMITDATE
     , NULLIF(V:"L_RECEIPTDATE"::VARCHAR, '')::VARCHAR   AS L_RECEIPTDATE
     , NULLIF(V:"L_SHIPINSTRUCT"::VARCHAR, '')::VARCHAR  AS L_SHIPINSTRUCT
     , NULLIF(V:"L_SHIPMODE"::VARCHAR, '')::VARCHAR      AS L_SHIPMODE
     , NULLIF(V:"L_COMMENT"::VARCHAR, '')::VARCHAR       AS L_COMMENT
FROM LINEITEMS
LIMIT 3;